# clone repos and change dir

In [ ]:
# if project does not exit, then clone the remote repo to local
# if do exit now, then pull from the remote, both the main project and submodules
import os
os.chdir("/content")

!if [ ! -d "/content/GraduationProject" ]; then git clone https://github.com/WenjayDu/GraduationProject.git && cd GraduationProject && git submodule update --init --recursive; else cd /content/GraduationProject && git pull && cd module_minc_keras && git checkout master && git pull && cd ../module_pocketflow && git checkout master && git pull; fi

os.chdir("/content/GraduationProject")


# prepare dataset

In [ ]:
# decompressing dataset output.tar.bz2
!tar -jxvf /content/GraduationProject/module_minc_keras/data/output.tar.bz2 -C /content/GraduationProject/datasets &> /dev/null
!mv /content/GraduationProject/datasets/output /content/GraduationProject/datasets/mri 

# prepare mri dataset
import os
os.chdir("/content/GraduationProject")
from data_processing.prepare_datasets import prepare_mri_dataset
# generate small datasets to avoid memory exceeding causing process killed on cloab
prepare_mri_dataset(ratios=[0.3, 0.15]) 


# construct unet with keras and train it

In [ ]:
!python3 /content/GraduationProject/unet_constructing/construct_unet_with_keras.py

## make prediction with trained model

In [ ]:
from module_minc_keras.predict import predict
predict(model_fn='ouput/keras_implementation/model_of_unet_at_mri.h5py', 
        predict_dir='ouput/keras_implementation/predictions',
        data_dir='datasets/mri_pad_4_results/data', 
        images_fn='datasets/mri_pad_4_results/report/images.csv', 
        loss='categorical_crossentropy',
        images_to_predict='all', 
        category="test", 
        verbose=True)


# construct unet with tensorflow and train it

In [ ]:
os.chdir("/content/GraduationProject")
!python3 ./unet_constructing/construct_unet_with_tf.py


# Use ngrok to map tensorboard to a public URL to enable us to visit

In [ ]:
# run tensorboard
LOG_DIR=LOGS_DIR
get_ipython().system_raw(
    'tensorboard --logdir {} --host 0.0.0.0 --port 6006 &'
    .format(LOG_DIR)
)


In [ ]:
# download and unzip ngrok
! wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
! unzip ngrok-stable-linux-amd64.zip


In [ ]:
# run ngrok
get_ipython().system_raw('./ngrok http 6006 &')


In [ ]:
# map to public URL
! curl -s http://localhost:4040/api/tunnels | \
    python3 -c "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])" | \
    xargs echo "the public url of tensorboard is -> "


# Freeze the graph to .pb file if necessary

In [ ]:
from data_processing.freeze_and_restore_graph import freeze_graph
freeze_graph()


# model compressing

In [ ]:
!if [ ! -d "/content/GraduationProject/module_pocketflow/model_compressing" ]; then cp -r /content/GraduationProject/model_compressing /content/GraduationProject/module_pocketflow/; fi 
os.chdir("/content/GraduationProject/module_pocketflow")
!./scripts/run_local.sh model_compressing/unet_at_mri_for_pf_run.py \
        --learner=full-prec \
        --cycle_length=1 \
        --nb_threads=1 \
        --buffer_size=1 \
        --prefetch_size=1